In [1]:
from collections import defaultdict

puzzle = defaultdict(int)

with open("inputs/day15-input.txt") as f:
    for key, value in enumerate(map(int, f.read().split(","))):
        puzzle[key] = value

## Part 1

In [2]:
%%time

from typing import Dict, List

class IntcodeComputer:
    def __init__(self, intcode: Dict[int, int]):
        self.intcode = intcode
        self.input = []

        
    def get_param_index(self, instruction_index: int, parameter: int) -> int:
        instruction = f"{self.intcode[instruction_index]:05d}"
        mode = instruction[-2 - parameter]
        is_position  = (mode == "0")
        is_immediate = (mode == "1")
        is_relative  = (mode == "2")
        
        param_index = instruction_index + parameter
        param_value = self.intcode[param_index]
        
        if is_position:
            return param_value
        
        elif is_immediate:
            return param_index
        
        elif is_relative:
            return self.relative_base + param_value
        
    def get_param(self, instruction_index: int, parameter: int) -> int:
        return self.intcode[self.get_param_index(instruction_index, parameter)]


    def get_opcode(self, instruction_index: int) -> int:
        opcode = str(self.intcode[instruction_index])[-2:]
        return int(opcode)


    def run_program(self) -> int:
        index = 0
        self.relative_base = 0
        
        while True:
            opcode = self.get_opcode(index)

            if opcode == 99:
                yield True
                break

            if opcode == 1:
                in1 = self.get_param(index, 1)
                in2 = self.get_param(index, 2)
                out_index = self.get_param_index(index, 3)

                self.intcode[out_index] = in1 + in2
                index += 4

            elif opcode == 2:
                in1 = self.get_param(index, 1)
                in2 = self.get_param(index, 2)
                out_index = self.get_param_index(index, 3)

                self.intcode[out_index] = in1 * in2
                index += 4

            elif opcode == 3:
                in1 = self.get_param_index(index, 1)

                self.intcode[in1] = self.input.pop(0)
                index += 2

            elif opcode == 4:
                in1 = self.get_param(index, 1)

                self.result = in1
                yield False
                index += 2

            elif opcode == 5:
                in1 = self.get_param(index, 1)
                if in1:
                    in2 = self.get_param(index, 2)
                    index = in2
                else:
                    index += 3

            elif opcode == 6:
                in1 = self.get_param(index, 1)
                if not in1:
                    in2 = self.get_param(index, 2)
                    index = in2
                else:
                    index += 3

            elif opcode == 7:
                in1 = self.get_param(index, 1)
                in2 = self.get_param(index, 2)
                out_index = self.get_param_index(index, 3)

                self.intcode[out_index] = int(in1 < in2)
                index += 4

            elif opcode == 8:
                in1 = self.get_param(index, 1)
                in2 = self.get_param(index, 2)
                out_index = self.get_param_index(index, 3)

                self.intcode[out_index] = int(in1 == in2)
                index += 4
                
            elif opcode == 9:
                in1 = self.get_param(index, 1)
                
                self.relative_base += in1
                index += 2
    

computer = IntcodeComputer(puzzle.copy())
program = computer.run_program()

inverse_direction = {
    0: 0,
    1: 2,
    2: 1,
    3: 4,
    4: 3
}

def backtrack(previous_direction: int, steps: int):
    for direction in range(1, 5):
        if direction == inverse_direction[previous_direction]:
            continue

        computer.input.append(direction)
        halted = next(program)
        
        if halted:
            return True
        
        status = computer.result
        if status == 0:
            continue
        elif status == 1:
            if backtrack(direction, steps + 1):
                return True
            computer.input.append(inverse_direction[direction])
            next(program)
        elif status == 2:
            print(steps)
            return True
        
backtrack(0, 1)

318
CPU times: user 95.9 ms, sys: 2.19 ms, total: 98.1 ms
Wall time: 102 ms


True

## Part 2

In [3]:
%%time

from typing import Tuple

direction_vectors = {
    1: (0, -1),
    2: (0, 1),
    3: (-1, 0),
    4: (1, 0)
}

def move(position, heading):
    direction = direction_vectors[heading]
    return (position[0] + direction[0], position[1] + direction[1])

area = {}
def backtrack_area(previous_direction: int, position: Tuple[int, int]):
    for direction in range(1, 5):
        if direction == inverse_direction[previous_direction]:
            continue

        computer.input.append(direction)
        halted = next(program)
        
        if halted:
            return True
        
        new_position = move(position, direction)
        status = computer.result
        if status == 0:
            area[new_position] = "#"
        elif status == 1:
            area[new_position] = "."
            backtrack_area(direction, new_position)
            computer.input.append(inverse_direction[direction])
            next(program)
        elif status == 2:
            area[new_position] = "O"
            backtrack_area(direction, new_position)
            computer.input.append(inverse_direction[direction])
            next(program)

computer = IntcodeComputer(puzzle.copy())
program = computer.run_program()
backtrack_area(0, (0, 0))

def print_area(area):
    min_x = min(area, key=lambda x: x[0])[0]
    min_y = min(area, key=lambda x: x[1])[1]
    max_x = max(area, key=lambda x: x[0])[0]
    max_y = max(area, key=lambda x: x[1])[1]
    
    for y in range(min_y, max_y + 1):
        for x in range(min_x, max_x + 1):
            print(area.get((x, y), " "), end="")
        print()
        
print_area(area)

update_positions = [x for x in area if area[x] == "O"]
minutes = 0
while update_positions:
    new_positions = []
    
    for position in update_positions:
        for heading in range(1, 5):
            new_position = move(position, heading)

            if area.get(new_position, "") == ".":
                new_positions.append(new_position)
                area[new_position] = "O"
                
    update_positions = new_positions
    minutes += 1

print(minutes - 1)

 ### ### ############# ##### ####### ### 
#...#...#.............#.....#.......#...#
#.#.#.#.#########.###.#.###.#####.#.###.#
#.#...#.........#.#.....#.#.......#.....#
#.#############.#.#######.#############.#
#...........#.....#...#.....#.......#...#
#.#########.###.###.#.#.#.###.#####.#.## 
#.#.......#...#.#...#.#.#.....#...#...#.#
#.#.#####.###.###.###.#.#######.#.#####.#
#...#...#.#.#.......#.#.........#.#O....#
 ######.#.#.#########.###########.#####.#
#.......#.#.....#.....#.........#.#...#.#
#.#.###.#.#.###.#.#####.#####.#.#.#.#.#.#
#.#...#.#...#...#.#...#.....#.#.#...#.#.#
#.###.#.###.#####.#.#.#####.#.#.#####.#.#
#...#.#...#.#.....#.#...#...#.#.....#.#.#
 ##.#.###.#.#.#####.#####.###.#.#####.#.#
#.#.#.#...#.#.....#.......#.#.#.#.....#.#
#.#.#.#####.#####.#######.#.#.###.#####.#
#.#.#.....#.#...#...#...#...#...#.#.....#
#.#.#####.#.#.#####.#.#.#.#####.#.#.###.#
#...#...#...#.....#.# #.#.#...#...#...#.#
#.###.#.#####.#.#.#.###.###.#.#.## ##.#.#
#.#...#...#...#.#.#...#.....#.#...